In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
# load functional definitions for activations like softmax
import torch.nn.functional as F
# print utilities with json
import json
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import timeit
%matplotlib inline

# 1. Quick intro: The simplicity of running an RNN in PyTorch

In [ ]:
# We instantiate an RNN layer with the RNN API from nn module
rnn_layer = nn.RNN(input_size=3, hidden_size=3, num_layers=1, batch_first=True)

# Make some fake data
fake_data = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [0, 1, 1]]
fake_data = torch.FloatTensor(fake_data)
print('fake data tensor size: ', fake_data.size())

### 1) Iterate with your own loop (yesss)

In [ ]:
H_t = []
h_t_1 = None
for x_t in fake_data:
    # turn into 3-D shape [batch_size, seq_len, feat_dim=3]
    x_t = Variable(x_t).view(1, -1, 3)
    h_t, h_t_1 = rnn_layer(x_t, h_t_1)
    H_t.append(h_t.data[0, 0, :].numpy())
H_t = np.array(H_t)
print(H_t)

### 2) Structure your data batching, and leave it to the framework (yesss too)

In [ ]:
h_t_1 = None
x_t = Variable(fake_data).view(1, -1, 3)
H_t, h_t_1 = rnn_layer(x_t, h_t_1)
print(H_t.data.numpy())

# 2. Going Deeper: Character prediction RNN-LSTM

#### We'll now proceed to read a text file from our local dir.

In [ ]:
MIN_CHARS=5
with open('aliceinwonderland.txt') as txt_f:
    # read the text file
    text = txt_f.read().lower()
# get sentences on line break (yes, a bit vague, but ok for this)
text_sents = list(filter(lambda x: len(x) > 0, text.split('\n')))
print('text corpus length in chars:', len(text))
print('total sentences: ', len(text_sents))

max_len = max(len(s) for s in text_sents)
min_len = min(len(s) for s in text_sents)
print('max sentence len: ', max_len)
print('min sentence len: ', min_len)

print('Example 10 sentences: ')
for i, sent in enumerate(text_sents[:10], start=1):
    print('{} >> {}'.format(i, sent))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# We'll print the char2idx to grasp how it looks like
print(json.dumps(char2idx, indent=2))

In [ ]:
# STATIC GRAPH PROCEDURE to inject data
# taken from Keras example https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
# ----------------------------------------------------------------
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2idx[char]] = 1
    y[i, char2idx[next_chars[i]]] = 1
print('Done vectorizing...')
print('X data tensor shape: ', X.shape)
print('y data tensor shape: ', y.shape)

**Note how we had to apply a certain maximum sequence length (40), and then we've built a tensor of input chunks of chars X, and a tensor of next chars x[t] named y**

### Gimme some PyTorch, please

Now we can unleash PyTorch capabilities to define the char RNN very simplistically, and process sequences **with true variable length!**

In [ ]:
# Make a sentence encoder helper function
def encode_sent(sent):
    x_seq = []
    for ch in sent:
        x_seq.append(char2idx[ch])
    return x_seq

In [ ]:
X = []
for i, sentence in enumerate(text_sents):
    # encode each sentence into its char integer code from char2idx dictionary
    X.append(encode_sent(sentence))
print('Resulting encoded sequences: ', len(X))
print('Example 3 encoded sequences: ')
for n, sent in enumerate(X[:3], start=1):
    print(sent)

### Building our char-RNN

In [ ]:
class charRNN(nn.Module):
    def __init__(self, char_vocab_size, emb_size=15,
                 rnn_size=128, rnn_layers=1, rnn_dropout=0.):
        super(charRNN, self).__init__()
        # build character Embedding layer
        self.emb = nn.Embedding(char_vocab_size, emb_size)
        self.rnn_layers = rnn_layers
        self.rnn_size = rnn_size
        # LSTM layers
        self.lstm = nn.LSTM(input_size=emb_size,
                            hidden_size=rnn_size,
                            num_layers=rnn_layers,
                            dropout=rnn_dropout,
                            batch_first=True,
                            bidirectional=False)
        # FC output layer into num classes (vocab size)
        self.fc = nn.Linear(rnn_size, char_vocab_size)
    
    def forward(self, seq, states=None):
        # input tensor is of shape [batch_size, seq_len]
        # it is a LongTensor containing an integer idx per char per seq
        assert len(seq.size()) == 2, seq.size()
        # project seq through embedding layer
        emb_ch = self.emb(seq)
        # emb_ch ~ [batch_size, seq_len, emb_size]
        H_t, states = self.lstm(emb_ch, states)
        # H_t ~ [batch_size, seq_len, rnn_size]
        # unroll tensor to 2-D to adjust to FC nature
        H_t_u = H_t.contiguous().view(-1, H_t.size(-1))
        # H_t_u ~ [batch_size x seq_len, rnn_size]
        y_t = F.log_softmax(self.fc(H_t_u))
        # return output predicted probs and rnn states
        return y_t, states

    def init_hidden_zero(self, curr_bsz):
        return (Variable(torch.zeros(self.rnn_layers, curr_bsz, self.rnn_size)),
                Variable(torch.zeros(self.rnn_layers, curr_bsz, self.rnn_size)))
        
# instance our lstm model
char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
print(char_lstm)

# HELPER FUNCTION WE'LL USE WE'LL SEE WHERE
def repackage_hidden(h):
    # https://github.com/pytorch/examples/blob/master/word_language_model/main.py
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [ ]:
# build a sentence decoder function to simplify our future calls to check predictions
def decode_pred_sent(pred_sent):
    dec_sent = ''
    for idx in pred_sent:
        dec_sent += idx2char[idx]
    return dec_sent

In [ ]:
# Make a text prediction function
def predict_text_from_seed(model, text_seed='alice was beginning ', num_preds=20):
    model.eval()
    # begin heating the lstm with some seed text and predict next num_preds chars
    enc_seed = encode_sent(text_seed)

    # first make the warm up to save into the LSTM states the seed
    states = None
    for code in enc_seed:
        # please note the Volatile keyword: we just want inference, not backprop
        code = Variable(torch.LongTensor([code]).view(1, 1), volatile=True)
        pred_ch, states = model(code, states)

    pred_ch = torch.max(pred_ch, dim=-1)[1]
    # |> symbol serves as a simple prompt to check where does it start the prediction
    resulting_pred = text_seed + '|>' + idx2char[pred_ch.data[0]]
    # Now iterate char by char in a feedback fashion
    for n in range(num_preds):
        pred_ch, states = model(pred_ch.view(1, 1), states)
        pred_ch = torch.max(pred_ch, dim=-1)[1]
        pred_idx = pred_ch.data[0]
        resulting_pred += idx2char[pred_idx]
    return resulting_pred

# predict some chars w/ RANDOM weights
print('resulting text: ', predict_text_from_seed(char_lstm, 'i found a door ', num_preds=100))

**Observe how we can iterate with external for loops to the LSTM structure, feeding back the data from time-step to time-step, contrary to TensorFlow methodology.**

### Training the model!

In [ ]:
def train_seqbyseq_epoch(X, curr_epoch, model, opt, log_freq, stateful=False):
    # A simple training loop sequence by sequence will serve our purposes
    # YES! THAT SIMPLE!!
    # specify we are in train mode (this will set Dropout/BN behaviors to train mode)
    model.train()
    states = model.init_hidden_zero(1)
    tr_losses = []
    for bidx, x in enumerate(X, start=1):
        # clean previous gradients in graph
        opt.zero_grad()
        # build input x and output y
        # output is just a shifted by 1 timestep version of x
        y = x[1:]
        x = x[:-1]
        # format the PyTorch Variables
        y = Variable(torch.LongTensor(y)).view(-1) # y ~ [seq_len]
        x = Variable(torch.LongTensor(x)).view(1, -1) # x ~ [1, seq_len]
        # (1) forward sequence
        if not stateful:
            pred, _ = model(x, states)
        else:
            pred, states = model(x, states)
            states = repackage_hidden(states)
            #raise NotImplementedError('Missing stateful implementation')
        
        # (2) compute loss: Negative Log Likelihood of correct classes
        loss = F.nll_loss(pred, y)
        # (3) backprop gradients
        loss.backward()
        # (4) update parameters
        opt.step()
        if bidx % log_freq == 0 or bidx >= len(X):
            tr_losses.append(loss.data[0])
            print('Batch {:4d}/{:4d} (epoch {:3d}) loss {:.3f}'.format(bidx,
                                                                       len(X),
                                                                       curr_epoch,
                                                                       tr_losses[-1]))
    return tr_losses
            
import timeit # time epochs

char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
NUM_EPOCHS=20
LOG_FREQ=400
# we can limit the samples in the dataset to speed up training (this is a toy example, remember)
max_samples = 100
tr_X = X[:max_samples]
opt = optim.Adam(char_lstm.parameters(), lr=0.001)
beg_t = timeit.default_timer()
tr_losses = []
for epoch in range(NUM_EPOCHS):
    # shuffle sentences
    shuffle(tr_X)
    tr_losses += train_seqbyseq_epoch(tr_X, epoch, char_lstm, opt, 400, False)
    end_t = timeit.default_timer()
    print('Elapsed time for epoch {:3d}: {:.3f} s'.format(epoch, end_t - beg_t))
    beg_t = end_t
    # see predictions change
    print('Epoch {} result {}'.format(epoch, '-' * 30))
    print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=30))

In [ ]:
print(predict_text_from_seed(char_lstm, 'she found a door ', num_preds=100))
print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=100))

In [ ]:
plt.title('Seqbyseq training curve')
plt.ylabel('NLLLoss')
plt.xlabel('Log tick of freq {}'.format(LOG_FREQ))
plt.plot(tr_losses)

### Well look at that learning noise... :( Let's smooth the learning with better updates

In [ ]:
def train_batches_epoch(X, batch_size, curr_epoch, model, opt, log_freq):
    # We still need batching with pytorch to smooth our training in the SGD training
    # BUT: Each batch can have a variable length in sequence dimension!!
    # specify we are in train mode (this will set Dropout/BN behaviors to train mode)
    model.train()
    
    n_batches = int(np.ceil(len(X) / batch_size))
    #print('Number of batches with batch_size {}:{}'.format(batch_size, n_batches))
    tr_losses = []
    for bidx, b_beg in enumerate(range(0, len(X), batch_size), start=1):
        left = min(len(X) - b_beg, batch_size)
        states = model.init_hidden_zero(left)
        # select batch of sequences of outputs and inputs
        x_batch = X[b_beg:b_beg + left]
        # get max_len and add PADDING to smaller seqs
        max_batch_len = max(len(s) for s in x_batch)
        min_batch_len = min(len(s) for s in x_batch)
        for s_idx in range(len(x_batch)):
            x_batch[s_idx] = [0] * (max_batch_len - len(x_batch[s_idx])) + x_batch[s_idx]
        x_batch = np.array(x_batch, dtype=np.int64)
        y_batch = x_batch[:, 1:]
        x_batch = x_batch[:, :-1]
        # clean previous gradients in graph
        opt.zero_grad()
        # format the PyTorch Variables
        y = Variable(torch.from_numpy(y_batch).contiguous()).view(-1) # y ~ [batch_size x seq_len]
        x = Variable(torch.from_numpy(x_batch).contiguous()).view(left, -1) # x ~ [batch_size, seq_len]
        # (1) forward sequence
        pred, _ = model(x, states)
        
        # (2) compute loss: Negative Log Likelihood of correct classes
        loss = F.nll_loss(pred, y)
        # (3) backprop gradients
        loss.backward()
        # (4) update parameters
        opt.step()
        if bidx % log_freq == 0 or bidx >= n_batches:
            tr_losses.append(loss.data[0])
            print('Batch {:4d}/{:4d} (epoch {:3d}) loss {:.3f}'.format(bidx,
                                                                       n_batches,
                                                                       curr_epoch,
                                                                       tr_losses[-1]))
    return tr_losses


char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
NUM_EPOCHS=20
LOG_FREQ=400
# we can limit the samples in the dataset to speed up training (this is a toy example, remember)
max_samples = 100
tr_X = X[:max_samples]
opt = optim.Adam(char_lstm.parameters(), lr=0.001)
beg_t = timeit.default_timer()
b_tr_losses = []
for epoch in range(NUM_EPOCHS):
    # shuffle sentences
    shuffle(tr_X)
    b_tr_losses += train_batches_epoch(tr_X, 10, epoch, char_lstm, opt, 400)
    end_t = timeit.default_timer()
    print('Elapsed time for epoch {:3d}: {:.3f} s'.format(epoch, end_t - beg_t))
    beg_t = end_t
    # see predictions change
    print('Epoch {} result {}'.format(epoch, '-' * 30))
    print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=30))

In [ ]:
plt.title('Batched training curve')
plt.ylabel('NLLLoss')
plt.xlabel('Log tick of freq {}'.format(LOG_FREQ))
plt.plot(b_tr_losses, label='batched learning')
plt.plot(tr_losses, label='sample-wise learning')
plt.legend()

### Last stage: limiting the max sequence length with statefulness

In [ ]:
def train_stateful_maxlen_batches_epoch(X, batch_size, max_len, curr_epoch, model, opt, log_freq, clip=0.):
    # specify we are in train mode (this will set Dropout/BN behaviors to train mode)
    model.train()
    # build a super long sequence out of all samples concatenated
    X_long = []
    for x in X:
        X_long += x
    # trim to multiple of seqlen x batch_size
    X_long = X_long[:batch_size * max_len * (len(X_long) // (batch_size * max_len))]
    X_arr = np.array(X_long, dtype=np.int64).reshape((-1, 1))
    X_arr = X_arr.reshape((batch_size, -1, 1))
    X_arr = np.split(X_arr, X_arr.shape[1] // max_len, axis=1)
    X_arr = np.concatenate(X_arr, axis=0)
    # print('X_arr shape: ', X_arr.shape)
    n_batches = int(np.ceil(X_arr.shape[0] / batch_size))
    # print('Number of batches with batch_size {}: {}'.format(batch_size, n_batches))
    tr_losses = []
    states = model.init_hidden_zero(batch_size)
    for bidx, b_beg in enumerate(range(0, X_arr.shape[0], batch_size), start=1):
        # select batch of sequences of outputs and inputs
        x_batch = X_arr[b_beg:b_beg + batch_size]
        x_batch = np.array(x_batch, dtype=np.int64)
        y_batch = x_batch[:, 1:]
        x_batch = x_batch[:, :-1]
        # clean previous gradients in graph
        opt.zero_grad()
        # format the PyTorch Variables
        y = Variable(torch.from_numpy(y_batch).contiguous()).view(-1) # y ~ [batch_size x seq_len]
        x = Variable(torch.from_numpy(x_batch).contiguous()).view(batch_size, -1) # x ~ [batch_size, seq_len]
        # (1) forward sequence
        pred, states = model(x, states)
        states = repackage_hidden(states)
        # (2) compute loss: Negative Log Likelihood of correct classes
        loss = F.nll_loss(pred, y)
        # (3) backprop gradients
        loss.backward()
        if clip > 0:
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            torch.nn.utils.clip_grad_norm(model.parameters(), clip)

        # (4) update parameters
        opt.step()
        if bidx % log_freq == 0 or bidx >= n_batches:
            tr_losses.append(loss.data[0])
            print('Batch {:4d}/{:4d} (epoch {:3d}) loss {:.3f}'.format(bidx,
                                                                       n_batches,
                                                                       curr_epoch,
                                                                       tr_losses[-1]))
    return tr_losses

In [ ]:
char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
NUM_EPOCHS=20
LOG_FREQ=400
# we can limit the samples in the dataset to speed up training (this is a toy example, remember)
max_samples = 100
tr_X = X[:max_samples]
opt = optim.Adam(char_lstm.parameters(), lr=0.001)
beg_t = timeit.default_timer()
st_tr_losses = []
for epoch in range(NUM_EPOCHS):
    # shuffle sentences
    shuffle(tr_X)
    st_tr_losses += train_stateful_maxlen_batches_epoch(tr_X, 10, 35, epoch, char_lstm, opt, 400)
    end_t = timeit.default_timer()
    print('Elapsed time for epoch {:3d}: {:.3f} s'.format(epoch, end_t - beg_t))
    beg_t = end_t
    # see predictions change
    print('Epoch {} result {}'.format(epoch, '-' * 30))
    print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=30))

In [ ]:
plt.title('Stateful Batched training curve')
plt.ylabel('NLLLoss')
plt.xlabel('Log tick of freq {}'.format(LOG_FREQ))
plt.plot(b_tr_losses, label='batched learning')
plt.plot(st_tr_losses, label='stateful batched learning')
plt.plot(tr_losses, label='sample-wise learning')
plt.legend()

In [ ]:
# Now max_len = 15, shorter to backprop less tsteps
char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
NUM_EPOCHS=20
LOG_FREQ=400
# we can limit the samples in the dataset to speed up training (this is a toy example, remember)
max_samples = 100
tr_X = X[:max_samples]
opt = optim.RMSprop(char_lstm.parameters(), lr=0.01)
beg_t = timeit.default_timer()
st_sh_tr_losses = []
for epoch in range(NUM_EPOCHS):
    # shuffle sentences
    shuffle(tr_X)
    st_sh_tr_losses += train_stateful_maxlen_batches_epoch(tr_X, 10, 15, epoch, char_lstm, opt, 400)
    end_t = timeit.default_timer()
    print('Elapsed time for epoch {:3d}: {:.3f} s'.format(epoch, end_t - beg_t))
    beg_t = end_t
    # see predictions change
    print('Epoch {} result {}'.format(epoch, '-' * 30))
    print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=30))

In [ ]:
plt.title('Stateful Batched training curve')
plt.ylabel('NLLLoss')
plt.xlabel('Log tick of freq {}'.format(LOG_FREQ))
plt.plot(b_tr_losses, label='batched learning')
plt.plot(st_tr_losses, label='stateful-35 batched learning')
plt.plot(st_sh_tr_losses, label='stateful-15 batched learning')
plt.plot(tr_losses, label='sample-wise learning')
plt.legend()

In [ ]:
# Now max_len = 15, shorter to backprop less tsteps
char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
NUM_EPOCHS=20
LOG_FREQ=400
# we can limit the samples in the dataset to speed up training (this is a toy example, remember)
max_samples = 100
tr_X = X[:max_samples]
opt = optim.RMSprop(char_lstm.parameters(), lr=0.01)
beg_t = timeit.default_timer()
st_sh_sh_tr_losses = []
for epoch in range(NUM_EPOCHS):
    # shuffle sentences
    shuffle(tr_X)
    st_sh_sh_tr_losses += train_stateful_maxlen_batches_epoch(tr_X, 10, 10, epoch, char_lstm, opt, 400)
    end_t = timeit.default_timer()
    print('Elapsed time for epoch {:3d}: {:.3f} s'.format(epoch, end_t - beg_t))
    beg_t = end_t
    # see predictions change
    print('Epoch {} result {}'.format(epoch, '-' * 30))
    print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=30))

In [ ]:
plt.title('Stateful Batched training curve')
plt.ylabel('NLLLoss')
plt.xlabel('Log tick of freq {}'.format(LOG_FREQ))
plt.plot(b_tr_losses, label='batched learning')
plt.plot(st_tr_losses, label='stateful-35 batched learning')
plt.plot(st_sh_tr_losses, label='stateful-15 batched learning')
plt.plot(st_sh_sh_tr_losses, label='stateful-10 batched learning')
plt.plot(tr_losses, label='sample-wise learning')
plt.legend()

In [ ]:
# Now max_len = 15, shorter to backprop less tsteps
char_lstm = charRNN(len(char2idx), rnn_size=100, rnn_dropout=0.)
NUM_EPOCHS=20
LOG_FREQ=400
# we can limit the samples in the dataset to speed up training (this is a toy example, remember)
max_samples = 100
tr_X = X[:max_samples]
opt = optim.RMSprop(char_lstm.parameters(), lr=0.01)
beg_t = timeit.default_timer()
st_cn_tr_losses = []
for epoch in range(NUM_EPOCHS):
    # shuffle sentences
    shuffle(tr_X)
    st_cn_tr_losses += train_stateful_maxlen_batches_epoch(tr_X, 10, 35, epoch, char_lstm, opt, 400, 0.2)
    end_t = timeit.default_timer()
    print('Elapsed time for epoch {:3d}: {:.3f} s'.format(epoch, end_t - beg_t))
    beg_t = end_t
    # see predictions change
    print('Epoch {} result {}'.format(epoch, '-' * 30))
    print(predict_text_from_seed(char_lstm, text_sents[1], num_preds=30))

In [ ]:
print(predict_text_from_seed(char_lstm, 'she found a door ', num_preds=50))
print(predict_text_from_seed(char_lstm, 'she went ', num_preds=50))
print(predict_text_from_seed(char_lstm, 'when ', num_preds=50))
print(predict_text_from_seed(char_lstm, 'a rabbit ', num_preds=50))
print(predict_text_from_seed(char_lstm, 'a golden key ', num_preds=50))

In [ ]:
plt.title('Stateful Batched training curve')
plt.ylabel('NLLLoss')
plt.xlabel('Log tick of freq {}'.format(LOG_FREQ))
plt.plot(b_tr_losses, label='batched learning')
plt.plot(st_tr_losses, label='stateful-35 batched learning')
plt.plot(st_cn_tr_losses, label='stateful-35 cnorm 0.2 batched learning')
plt.plot(st_sh_tr_losses, label='stateful-15 batched learning')
plt.plot(st_sh_sh_tr_losses, label='stateful-10 batched learning')
plt.plot(tr_losses, label='sample-wise learning')
plt.legend()

### Proposals to do 

* Change the dataset into another txt file
* Introduce a sequence length filtering in model forward, to **exclude padding positions** in sequence when batching with padding
* Implement a noisy initial hidden state with normal distribution ~ N(0, I)
* Implement a **learnable initial hidden state** (not just zero vector)
* Implement the option to use GRU layers instead of LSTM in the model (BEWARE with cell differences, like states)
* Implement the option to use RNN layers instead of LSTM in the model (BEWARE with cell differences, like states)
* Compare the **performance of RNN vs GRU vs LSTM**, training for 20 epochs with the desired amount of data
* Implement **stateful batched trainer** method (making sample i-th from batch b-th continue in sample i-th from batch (b+1)-th